# References:
[https://medium.com/tensorflow/tensorflow-model-optimization-toolkit-pruning-api-42cac9157a6a](http://)
[https://towardsdatascience.com/pruning-deep-neural-network-56cae1ec5505](http://)
[https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras](http://)
[https://www.tensorflow.org/model_optimization/guide/quantization/training](http://)
[https://www.tensorflow.org/model_optimization/guide/quantization/post_training](http://)
[https://medium.com/apache-mxnet/model-quantization-for-production-level-neural-network-inference-f54462ebba05](http://)


# **Pruning**

Pruning is one of the methods for inference to efficiently produce models smaller in size, more memory-efficient, more power-efficient and faster at inference with minimal loss in accuracy, other such techniques being weight sharing and quantization.

**why Pruning ?**
As we Know the state of the art models are getting more and more accurate but this progress comes with a cost of different resources.
1. Models are getting larger
2. The Second Challenge: Speed
3. The Third Challenge: Energy Efficiency

**Solution — Efficient Inference Algorithms**
1. Weight pruning
2. Quantization

 **Weight pruning:**
 In Weight Pruning our optimizations aims to reduce the number of parameters and operations involved in the computation by removing connections, and thus parameters, in between neural network layers.
![](http:////miro.medium.com/max/875/0*iNI8Oc80Eunm8NgI)

Weight pruning means eliminating unnecessary values in the weight tensors. We are practically setting the neural network parameters’ values to zero to remove what we estimate are unnecessary connections between the layers of a neural network. This is done during the training process to allow the neural network to adapt to the changes.
Here, to achieve sparsity of k% we rank the individual weights in weight matrix W according to their magnitude, and then set to zero the smallest k%.

**Results across several models**
In our experiments, we have validated that this technique can be successfully applied to different types of models across distinct tasks, from image processing convolutional-based neural networks to speech processing ones using recurrent neural networks. The following table shows a subset of some of these experimental results.

![](http:////miro.medium.com/max/875/1*GxXn3vZBtnYS3SuUf_-7AA.png)

**Quantization:**
There are two forms of quantization: post-training quantization and quantization aware training. Start with post-training quantization since it's easier to use, though quantization aware training is often better for model accuracy.

 **Post-training quantization:**
Post-training quantization includes general techniques to reduce CPU and hardware accelerator latency, processing, power, and model  size with little degradation in model accuracy. These techniques can be performed on an already-trained float TensorFlow model and applied during TensorFlow Lite conversion. These techniques are enabled as options in the TensorFlow Lite converter.

**quantization aware training:**
Quantization aware training emulates inference-time quantization, creating a model that downstream tools will use to produce actually quantized models. The quantized models use lower-precision (e.g. 8-bit instead of 32-bit float), leading to benefits during deployment.

In [1]:
 !pip install -q tensorflow-model-optimization

In [2]:
import tempfile
import os
import tensorflow as tf
import numpy as np
from tensorflow import keras
%load_ext tensorboard

**Train a model for MNIST without pruning**

In [3]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  train_images,
  train_labels,
  epochs=4,
  validation_split=0.1,
)

11493376/11490434 [==============================] - 0s 0us/step
Epoch 1/4
1688/1688 [==============================] - 5s 3ms/step - loss: 0.2978 - accuracy: 0.9158 - val_loss: 0.1170 - val_accuracy: 0.9685
Epoch 2/4
1688/1688 [==============================] - 4s 3ms/step - loss: 0.1188 - accuracy: 0.9666 - val_loss: 0.0874 - val_accuracy: 0.9750
Epoch 3/4
1688/1688 [==============================] - 4s 3ms/step - loss: 0.0877 - accuracy: 0.9751 - val_loss: 0.0683 - val_accuracy: 0.9815
Epoch 4/4
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0718 - accuracy: 0.9787 - val_loss: 0.0670 - val_accuracy: 0.9832


**Evaluate baseline test accuracy and save the model for later usage.**

In [4]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)


Baseline test accuracy: 0.977400004863739
Saved baseline model to: /tmp/tmpbygoi1vi.h5


**Fine-tune pre-trained model with pruning**

Define the model:

You will apply pruning to the whole model and see this in the model summary.
In this example, you start the model with 50% sparsity (50% zeros in weights) and end with 80% sparsity.

In [5]:
import tensorflow_model_optimization as tfmot
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 
num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs
# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}
model_for_pruning = prune_low_magnitude(model, **pruning_params)
# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_reshape  (None, 28, 28, 1)         1         
_________________________________________________________________
prune_low_magnitude_conv2d ( (None, 26, 26, 12)        230       
_________________________________________________________________
prune_low_magnitude_max_pool (None, 13, 13, 12)        1         
_________________________________________________________________
prune_low_magnitude_flatten  (None, 2028)              1         
_________________________________________________________________
prune_low_magnitude_dense (P (None, 10)                40572     
Total params: 40,805
Trainable params: 20,410
Non-trainable params: 20,395
_________________________________________________________________


**Train and evaluate the model against baseline**

Fine tune with pruning for two epochs.

tfmot.sparsity.keras.UpdatePruningStep is required during training, and tfmot.sparsity.keras.PruningSummaries provides logs for tracking progress and debugging.

In [6]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]
  
model_for_pruning.fit(train_images, train_labels,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

Epoch 1/2
422/422 [==============================] - 3s 8ms/step - loss: 0.0866 - accuracy: 0.9763 - val_loss: 0.1083 - val_accuracy: 0.9745
Epoch 2/2
422/422 [==============================] - 4s 8ms/step - loss: 0.1213 - accuracy: 0.9696 - val_loss: 0.0922 - val_accuracy: 0.9755


**For this example, there is minimal loss in test accuracy after pruning, compared to the baseline.**

In [7]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)

Baseline test accuracy: 0.977400004863739
Pruned test accuracy: 0.968500018119812


**The logs show the progression of sparsity on a per-layer basis.**

In [8]:
#%tensorboard --logdir={logdir}

**Create 3x smaller models from pruning**

Both tfmot.sparsity.keras.strip_pruning and applying a standard compression algorithm (e.g. via gzip) are necessary to see the compression benefits of pruning.

First, create a compressible model for TensorFlow.

In [9]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

Saved pruned Keras model to: /tmp/tmpu9shlg_8.h5


**Then, create a compressible model for TFLite.**

In [10]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)

Saved pruned TFLite model to: /tmp/tmp7s_okufe.tflite


**Define a helper function to actually compress the models via gzip and measure the zipped size.**

In [11]:
def get_gzipped_model_size(file):
    # Returns size of gzipped model, in bytes.
    import os
    import zipfile
    _, zipped_file = tempfile.mkstemp('.zip')
    with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
        f.write(file)

    return os.path.getsize(zipped_file)

**Compare and see that the models are 3x smaller from pruning.**

In [12]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

Size of gzipped baseline Keras model: 77986.00 bytes
Size of gzipped pruned Keras model: 25655.00 bytes
Size of gzipped pruned TFlite model: 24564.00 bytes


# Create a 10x smaller model from combining pruning and quantization

**You can apply post-training quantization to the pruned model for additional benefits.**

In [13]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()

_, quantized_and_pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(quantized_and_pruned_tflite_file, 'wb') as f:
  f.write(quantized_and_pruned_tflite_model)

print('Saved quantized and pruned TFLite model to:', quantized_and_pruned_tflite_file)

print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_pruned_tflite_file)))

Saved quantized and pruned TFLite model to: /tmp/tmpp5rmvqsk.tflite
Size of gzipped baseline Keras model: 77986.00 bytes
Size of gzipped pruned and quantized TFlite model: 7759.00 bytes


**See persistence of accuracy from TF to TFLite**

**Define a helper function to evaluate the TF Lite model on the test dataset.**

In [14]:
import numpy as np

def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on ever y image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

**You evaluate the pruned and quantized model and see that the accuracy from TensorFlow persists to the TFLite backend.**

In [15]:
interpreter = tf.lite.Interpreter(model_content=quantized_and_pruned_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('Pruned and quantized TFLite test_accuracy:', test_accuracy)
print('Pruned TF test accuracy:', model_for_pruning_accuracy)

Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.
Evaluated on 8000 results so far.
Evaluated on 9000 results so far.


Pruned and quantized TFLite test_accuracy: 0.9683
Pruned TF test accuracy: 0.968500018119812


# Conclusion

In this tutorial, we saw how to create sparse models with the TensorFlow Model Optimization Toolkit API for both TensorFlow and TFLite. we then combined pruning with post-training quantization for additional benefits.

We created a 10x smaller model for MNIST, with minimal accuracy difference.